In [1]:
import pandas as pd
import sqlite3

## INSTANCIA DE BBDD ##


In [2]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/Bar_TB.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

## FUNCION PARA INTERACTUAR CON LAS BASES DE DATOS

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## CREAR LAS TABLAS 

In [10]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

cerveza
bares
empleados
reparto


### Tabla cerveza

In [ ]:
query_cerveza = """
CREATE TABLE cerveza (
    CodC VARCHAR (3),
    Envase VARCHAR (50),
    Capacidad FLOAT,
    Stock INT
)
"""

crsr.execute(query_cerveza)

### Tabla bares

In [ ]:
query_bares = """
CREATE TABLE bares (
    CodB VARCHAR (4),
    Nombre VARCHAR (50),
    Cif VARCHAR (10),
    Localidad VARCHAR (50)
)
"""

crsr.execute(query_bares)

### Tabla empleados

In [ ]:
query_empleados = """
CREATE TABLE empleados (
    CodE VARCHAR (2),
    Nombre VARCHAR (50),
    Sueldo INT 
)
"""

crsr.execute(query_empleados)

### Tabla reparto


In [ ]:
query_reparto = """
CREATE TABLE reparto (
    CodE VARCHAR (2),
    CodB VARCHAR (4),
    CodC VARCHAR (3),
    Fecha DATE,
    Cantidad INT 
)
"""

crsr.execute(query_reparto)

### INSERTAR DATOS

In [11]:
# Insertamos datos en la tabla cerveza
query_insert = """
INSERT INTO cerveza VALUES
('01', 'Botella', 0.2, 3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04', 'Botella', 1, 288),
('05', 'Barril', 60, 30)
"""
crsr.execute (query_insert)

In [ ]:
pd.read_sql ('SELECT * FROM cerveza', connection)

In [52]:
# Insertamos datos en la tabla reparto
query_insert2 = """
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) VALUES
('1', '001', '01', '2005-10-21', 240),
('1', '001', '02', '2005-10-21', 48),
('1', '002', '03', '2005-10-22', 60),
('1', '004', '05', '2005-10-22', 4),
('2', '002', '03', '2005-10-22', 48),
('2', '002', '05', '2005-10-23', 2),
('2', '004', '01', '2005-10-23', 480),
('2', '004', '02', '2005-10-24', 72),
('3', '003', '03', '2005-10-24', 48),
('3', '003', '04', '2005-10-25', 20)
"""
crsr.execute (query_insert)

In [ ]:
pd.read_sql('SELECT * FROM reparto', connection)

In [ ]:
#pd.read_sql('DELETE * FROM reparto', connection)

In [63]:
# Insertamos datos en la tabla empleados
empleados = pd.DataFrame ({
                        "CodE" : ["1", "2", "3"],
                        "Nombre" : ["Prudencio Caminero", "Vicente Merario", "Valentin Siempre" ],
                        "Sueldo" : [120000, 110000, 10000],
                        })


empleados.to_sql(name = 'empleados', con = connection, if_exists='append', index=False)

3

In [64]:
# Insertamos datos en la tabla bares
bares = pd.DataFrame ({
                        "CodB" : ["001", "002", "003", "004"],
                        "Nombre" : ["Stop", "Las Vegas", "Club Social", "Otra Ronda"],
                        "Cif" : ["11111111X", "22222222Y", None, "33333333Z"],
                        "Localidad" : ["Villa Botijo", "Villa Botijo", "Las Ranas", "La Esponja"]

                        })
bares

bares.to_sql(name = 'bares', con = connection, if_exists='append', index=False)

4